In [29]:
import os
import mido
from pathlib import Path
from mido import MidiFile, MidiTrack, merge_tracks

In [3]:
mid_file = 'Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep.mid'

In [4]:
mid = mido.MidiFile("../data/raw/" + mid_file)

## COMBINE MIDI TRACKS

In [6]:
def combine_midi_tracks(mid, output_path="../data/processed/"):
    if not isinstance(mid, MidiFile):
        raise ValueError("Expected a MidiFile object as input")
    
    combined_midi = MidiFile(ticks_per_beat=mid.ticks_per_beat)

    combined_midi.tracks = [merge_tracks(mid.tracks)]

    output_file = os.path.join(output_path, Path(mid.filename).stem + "_merged.mid")
    combined_midi.save(output_file)

In [7]:
def split_midi_file(mid, output_path="../data/processed/"):
    if not isinstance(mid, MidiFile):
        raise ValueError("Expected a MidiFile object as input")
    
    split_midi = MidiFile()
    split_midi.ticks_per_beat = mid.ticks_per_beat
    for msg in mid: 
        if not msg.is_meta:
            split_midi.tracks.append([msg])

In [8]:
print(mid.ticks_per_beat)
print(type(mid))
print(len(mid.tracks))
combine_midi_tracks(mid)

48
<class 'mido.midifiles.midifiles.MidiFile'>
9


## Split by bars

In [9]:
mid_file = "Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged.mid"
mid_unify = mido.MidiFile("../data/processed/" + mid_file)

In [43]:
BARS_TO_EXTRACT = 4

In [98]:
def save_bar_messages(message, bar_number, ticks_per_beat, filename, output_path="../data/processed/"):
    bar_midi = MidiFile(ticks_per_beat=ticks_per_beat)
    bar_track = MidiTrack()
    bar_midi.tracks.append(bar_track)
    for msg in message:
        msg.time = msg.time # int(msg.time)
        bar_track.append(msg)
    output_file = os.path.join(output_path, f"{filename}_{bar_number}.mid")
    bar_midi.save(output_file)
    print(f"Bar {bar_number} saved to {output_file}")

In [64]:
metadata_info = [msg for msg in mid_unify if msg.is_meta]
metadata_info[:-1]
metadata_info[-1]

MetaMessage('end_of_track', time=0)

In [99]:
# ANTIGUO -> funciona, pero va mal que no coge bien los nuevos acordes
def split_midi_by_bar(mid, bars_to_extract = 4, path="data/processed"):
    filename = Path(mid.filename).stem

    ticks_per_beat = mid.ticks_per_beat
    numerator_time_signature = [msg.numerator for msg in mid_unify if msg.type == 'time_signature'][0]
    ticks_per_bar = ticks_per_beat * numerator_time_signature
    ticks_to_extract = ticks_per_bar * bars_to_extract

    current_bar = 0
    current_ticks = 0
    metadata_info = [msg for msg in mid_unify if msg.is_meta]
    bar_messages = []

    for track in mid.tracks: 
        for msg in track:
            current_ticks += msg.time
            if current_ticks >= ticks_to_extract:
                current_bar += 1
                current_ticks -= ticks_to_extract
                bar_messages.append(metadata_info[-1]) # add the last meta message
                save_bar_messages(bar_messages, current_bar, ticks_per_beat, filename)
                bar_messages = metadata_info[:-1] # exclude the last meta message
                bar_messages.append(msg)
            bar_messages.append(msg)
    
    if bar_messages:
        current_bar += 1
        save_bar_messages(bar_messages, current_bar, ticks_per_beat, filename)

In [78]:
split_midi_by_bar(mid_unify, BARS_TO_EXTRACT)

Bar 1 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_1.mid
Bar 2 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_2.mid
Bar 3 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_3.mid
Bar 4 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_4.mid
Bar 5 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_5.mid
Bar 6 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_6.mid
Bar 7 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_7.mid
Bar 8 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_8.mid
Bar 9 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_9.mid
Bar 10 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_10.mid
Bar 11 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_11.mid
Bar 12

In [107]:
# BUENO
def split_midi_by_bar(mid, bars_to_extract = 4, path="data/processed"):
    filename = Path(mid.filename).stem

    ticks_per_beat = mid.ticks_per_beat
    numerator_time_signature = [msg.numerator for msg in mid_unify if msg.type == 'time_signature'][0]
    ticks_per_bar = ticks_per_beat * numerator_time_signature
    ticks_to_extract = ticks_per_bar * 4

    current_bar = 0
    current_ticks = 0
    metadata_info = [msg for msg in mid_unify if msg.is_meta]
    bar_messages = []
    chord_messages = []

    for track in mid.tracks:
        for msg in track:
            if msg.time == 0:
                chord_messages.append(msg)
            else: 
                chord_messages.append(msg)
                # CASE 1: the chord is of the next section
                if current_ticks + msg.time >= ticks_to_extract:
                    current_bar += 1
                    current_ticks = 0
                    bar_messages.append(metadata_info[-1])
                    save_bar_messages(bar_messages, current_bar, ticks_per_beat, filename)
                    bar_messages = metadata_info[:-1]
                    bar_messages.extend(chord_messages)
                    chord_messages = []
                # CASE 2: the chord is of the current section
                else:
                    current_ticks += msg.time
                    bar_messages.extend(chord_messages)
                    chord_messages = []

In [108]:
mid = mido.MidiFile("../data/processed/" + mid_file)
split_midi_by_bar(mid, BARS_TO_EXTRACT)

Bar 1 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_1.mid
Bar 2 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_2.mid
Bar 3 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_3.mid
Bar 4 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_4.mid
Bar 5 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_5.mid
Bar 6 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_6.mid
Bar 7 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_7.mid
Bar 8 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_8.mid
Bar 9 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_9.mid
Bar 10 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_10.mid
Bar 11 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_11.mid
Bar 12

In [106]:
mid = mido.MidiFile("../data/processed/" + mid_file)
filename = Path(mid.filename).stem

ticks_per_beat = mid.ticks_per_beat
numerator_time_signature = [msg.numerator for msg in mid_unify if msg.type == 'time_signature'][0]
ticks_per_bar = ticks_per_beat * numerator_time_signature
ticks_to_extract = ticks_per_bar * 4

current_bar = 0
current_ticks = 0
metadata_info = [msg for msg in mid_unify if msg.is_meta]
bar_messages = []
chord_messages = []

for track in mid.tracks:
    for msg in track:
        if msg.time == 0:
            chord_messages.append(msg)
        else: 
            chord_messages.append(msg)
            # CASE 1: the chord is of the next section
            if current_ticks + msg.time >= ticks_to_extract:
                current_bar += 1
                current_ticks = 0
                bar_messages.append(metadata_info[-1])
                save_bar_messages(bar_messages, current_bar, ticks_per_beat, filename)
                bar_messages = metadata_info[:-1]
                bar_messages.extend(chord_messages)
                chord_messages = []
            # CASE 2: the chord is of the current section
            else:
                current_ticks += msg.time
                bar_messages.extend(chord_messages)
                chord_messages = []

Bar 1 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_1.mid
Bar 2 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_2.mid
Bar 3 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_3.mid
Bar 4 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_4.mid
Bar 5 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_5.mid
Bar 6 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_6.mid
Bar 7 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_7.mid
Bar 8 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_8.mid
Bar 9 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_9.mid
Bar 10 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_10.mid
Bar 11 saved to ../data/processed/Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_11.mid
Bar 12

In [79]:
mid_test_1 = mido.MidiFile("../data/processed/" + "Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_1.mid")
mid_test_1

MidiFile(type=1, ticks_per_beat=48, tracks=[
  MidiTrack([
    MetaMessage('set_tempo', tempo=500000, time=0),
    MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    Message('program_change', channel=0, program=0, time=0),
    Message('note_on', channel=0, note=63, velocity=127, time=0),
    Message('program_change', channel=1, program=0, time=0),
    Message('note_on', channel=1, note=60, velocity=127, time=0),
    Message('program_change', channel=2, program=0, time=0),
    Message('note_on', channel=2, note=55, velocity=127, time=0),
    Message('note_off', channel=0, note=63, velocity=0, time=96),
    Message('note_on', channel=0, note=62, velocity=127, time=0),
    Message('note_off', channel=1, note=60, velocity=0, time=0),
    Message('note_on', channel=1, note=59, velocity=127, time=0),
    Message('note_off', channel=2, note=55, velocity=0, time=0),
    Message('note_on', channel=2, note=54, velocity=127, 

In [81]:
mid_test_2 = mido.MidiFile("../data/processed/" + "Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_2.mid")
mid_test_2

MidiFile(type=1, ticks_per_beat=48, tracks=[
  MidiTrack([
    MetaMessage('set_tempo', tempo=500000, time=0),
    MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    Message('note_off', channel=0, note=68, velocity=0, time=48),
    Message('note_off', channel=0, note=68, velocity=0, time=48),
    Message('note_on', channel=0, note=69, velocity=127, time=0),
    Message('note_off', channel=1, note=63, velocity=0, time=0),
    Message('note_on', channel=1, note=64, velocity=127, time=0),
    Message('note_off', channel=2, note=60, velocity=0, time=0),
    Message('note_on', channel=2, note=61, velocity=127, time=0),
    Message('note_off', channel=0, note=69, velocity=0, time=96),
    Message('note_on', channel=0, note=67, velocity=127, time=0),
    Message('note_off', channel=1, note=64, velocity=0, time=0),
    Message('note_on', channel=1, note=63, velocity=127, time=0),
    Message('note_off', channel=2, note=61,

In [80]:
mid_test_14 = mido.MidiFile("../data/processed/" + "Banjo-Kazooie_N64_Banjo-Tooie_Cauldron Keep_merged_14.mid")
mid_test_14

MidiFile(type=1, ticks_per_beat=48, tracks=[
  MidiTrack([
    MetaMessage('set_tempo', tempo=500000, time=0),
    MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    Message('note_off', channel=0, note=60, velocity=0, time=24),
    Message('note_off', channel=0, note=60, velocity=0, time=24),
    Message('note_on', channel=0, note=61, velocity=127, time=0),
    Message('note_off', channel=2, note=56, velocity=0, time=0),
    Message('note_on', channel=2, note=57, velocity=127, time=0),
    Message('note_off', channel=3, note=48, velocity=0, time=0),
    Message('note_on', channel=3, note=49, velocity=127, time=0),
    Message('note_off', channel=4, note=63, velocity=0, time=0),
    Message('note_on', channel=4, note=64, velocity=127, time=0),
    Message('note_off', channel=6, note=75, velocity=0, time=0),
    Message('note_on', channel=6, note=69, velocity=127, time=0),
    Message('note_off', channel=7, note=51, 